In [11]:
import pandas as pd
import numpy as np
import joblib
import os

In [58]:
# Load your test set
X_test_binary = pd.read_csv(r"C:\Users\ghaza\Desktop\FYP\Application\AI\data\dropped_final_training\X_test_dropped.csv")
X_test_multi = pd.read_csv(r"C:\Users\ghaza\Desktop\FYP\Application\AI\data\dropped_final_training\X_test_multi_dropped.csv")

In [60]:
X_test_binary.head()

,Water_Flow_Rate,Temperature_Fluctuation_Index,Pressure_Stability_Index,Detergent_Level,Hydraulic_Pressure,Coolant_Temperature
0,-0.109566,-0.912109,-0.354486,-0.317797,-0.198730,1.252999
1,-0.460948,-0.123897,-0.511168,-0.842531,-0.198730,-0.102126
2,-0.210826,-0.247584,-0.250963,-0.387861,-0.741503,0.779682
3,-1.120970,1.359175,0.749614,0.270374,0.494807,0.955652
4,1.107329,-0.418114,0.056019,-0.992408,1.667591,-0.196436


In [62]:
X_test_multi.head()

,Water_Flow_Rate,Temperature_Fluctuation_Index,Pressure_Stability_Index,Detergent_Level,Hydraulic_Pressure,Hydraulic_Oil_Temperature
0,-0.109566,-0.912109,-0.354486,-0.317797,-0.198730,0.247529
1,-0.460948,-0.123897,-0.511168,-0.842531,-0.198730,-1.119986
2,-0.210826,-0.247584,-0.250963,-0.387861,-0.741503,-0.373477
3,-1.120970,1.359175,0.749614,0.270374,0.494807,-0.395548
4,1.107329,-0.418114,0.056019,-0.992408,1.667591,-0.055879


In [64]:
model_binary = joblib.load(r"C:\Users\ghaza\Desktop\FYP\Application\AI\models\final_binary_soft_voting_ensemble.pkl")
model_binary

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(min_samples_leaf=5,
                                                     n_estimators=200,
                                                     random_state=42)),
                             ('mlp',
                              MLPClassifier(activation='tanh',
                                            early_stopping=True,
                                            hidden_layer_sizes=(100, 50),
                                            max_iter=300, random_state=42)),
                             ('gb',
                              GradientBoostingClassifier(learning_rate=0.2,
                                                         max_depth=7,
                                                         n_estimators=200,
                                                         n_iter_no_change=10,
                                                         random_state=42))],
                 voting='soft')

In [66]:
joblib.dump(model_binary, r"C:\Users\ghaza\Desktop\FYP\Application\AI\models\final_binary_soft_voting_ensemble_compat.pkl", protocol=4)

['C:\\Users\\ghaza\\Desktop\\FYP\\Application\\AI\\models\\final_binary_soft_voting_ensemble_compat.pkl']

In [16]:
model_multiclass = joblib.load(r"C:\Users\ghaza\Desktop\FYP\Application\AI\models\final_multi_ensemble.pkl")
model_multiclass

ModuleNotFoundError: No module named '_loss'

In [ ]:
joblib.dump(model_multiclass, r"C:\Users\ghaza\Desktop\FYP\Application\AI\models\final_multi_soft_voting_ensemble_compat.pkl", protocol=4)

In [72]:
# ====== Step 1: Predict using binary model (e.g. machine_failure yes/no) ======
y_pred_binary = model_binary.predict(X_test_binary)

# ====== Step 2: Predict using multiclass model (e.g. failure_type) ======
y_pred_multiclass = model_multiclass.predict(X_test_multi)

In [73]:
# ====== Step 3: Track inconsistency ======
# Logic: If binary model says "No Failure" (0), then multiclass should say "No Failure" (1)
inconsistent = ((y_pred_binary == 0) & (y_pred_multiclass != 1)).sum()

# Total comparisons for context
total = len(y_pred_binary)
consistency_rate = 100 * (1 - inconsistent / total)

print(f"Inconsistent predictions (binary=0, multiclass≠1): {inconsistent}")
print(f"Consistency rate: {consistency_rate:.2f}%")

Inconsistent predictions (binary=0, multiclass≠1): 3752
Consistency rate: 92.85%


In [8]:
import joblib
import numpy as np

print("=== FIXING ALL MODELS WITH PROTOCOL 4 ===")

# Your exact paths
binary_path = r"C:\Users\ghaza\Desktop\FYP\Application\AI\models\final_binary_soft_voting_ensemble_compat.pkl"
multi_path = r"C:\Users\ghaza\Desktop\FYP\Application\AI\models\final_multi_soft_voting_ensemble_compat.pkl"
scaler_binary_path = r"C:\Users\ghaza\Desktop\FYP\Application\AI\models\scaler_binary_6.pkl"
scaler_multi_path = r"C:\Users\ghaza\Desktop\FYP\Application\AI\models\scaler_multi_6.pkl"
label_encoder_path = r"C:\Users\ghaza\Desktop\FYP\Application\AI\models\label_encoder_6.pkl"

try:
    print("1. Loading binary model...")
    model_binary = joblib.load(binary_path)
    print("   ✅ Binary model loaded")
    
    print("2. Loading multi model...")
    model_multi = joblib.load(multi_path)
    print("   ✅ Multi model loaded")
    
    print("3. Loading binary scaler...")
    scaler_binary = joblib.load(scaler_binary_path)
    print("   ✅ Binary scaler loaded")
    
    print("4. Loading multi scaler...")
    scaler_multi = joblib.load(scaler_multi_path)
    print("   ✅ Multi scaler loaded")
    
    print("5. Loading label encoder...")
    label_encoder = joblib.load(label_encoder_path)
    print("   ✅ Label encoder loaded")
    
    print("\n=== TESTING BEFORE RESAVING ===")
    
    # Test multi model
    dummy_input = np.array([[10.0, 8.0, 15.0, 49.0, 4900.0, 39.0]])
    multi_proba = model_multi.predict_proba(dummy_input)[0]
    print(f"Multi model classes: {len(multi_proba)}")
    print(f"Label encoder classes: {len(label_encoder.classes_)}")
    print(f"Classes: {label_encoder.classes_}")
    
    if len(multi_proba) != 5:
        print(f"❌ PROBLEM: Multi model outputs {len(multi_proba)} classes, expected 5")
    else:
        print("✅ Multi model has correct number of classes")
    
    print("\n=== RESAVING WITH PROTOCOL 4 ===")
    
    print("Resaving binary model...")
    joblib.dump(model_binary, binary_path, protocol=4)
    print("   ✅ Binary model saved")
    
    print("Resaving multi model...")
    joblib.dump(model_multi, multi_path, protocol=4)
    print("   ✅ Multi model saved")
    
    print("Resaving binary scaler...")
    joblib.dump(scaler_binary, scaler_binary_path, protocol=4)
    print("   ✅ Binary scaler saved")
    
    print("Resaving multi scaler...")
    joblib.dump(scaler_multi, scaler_multi_path, protocol=4)
    print("   ✅ Multi scaler saved")
    
    print("Resaving label encoder...")
    joblib.dump(label_encoder, label_encoder_path, protocol=4)
    print("   ✅ Label encoder saved")
    
    print("\n=== TESTING AFTER RESAVING ===")
    
    # Reload and test
    model_binary_new = joblib.load(binary_path)
    model_multi_new = joblib.load(multi_path)
    scaler_binary_new = joblib.load(scaler_binary_path)
    scaler_multi_new = joblib.load(scaler_multi_path)
    label_encoder_new = joblib.load(label_encoder_path)
    
    print("✅ All models reload successfully!")
    
    # Test with your Case 11 data
    print("\n=== TESTING WITH CASE 11 DATA ===")
    case_11_data = {
        "water_flow_rate": 25.759390,
        "temperature_fluctuation_index": 16.297520,
        "pressure_stability_index": 54.846547,
        "detergent_level": 56.931079,
        "hydraulic_pressure": 1302.737141,
        "coolant_temperature": 74.091336,
        "hydraulic_oil_temperature": 46.427831
    }
    
    # Binary input
    binary_input = np.array([[
        case_11_data["water_flow_rate"],
        case_11_data["temperature_fluctuation_index"],
        case_11_data["pressure_stability_index"],
        case_11_data["detergent_level"],
        case_11_data["hydraulic_pressure"],
        case_11_data["coolant_temperature"]
    ]])
    
    # Multi input
    multi_input = np.array([[
        case_11_data["water_flow_rate"],
        case_11_data["temperature_fluctuation_index"],
        case_11_data["pressure_stability_index"],
        case_11_data["detergent_level"],
        case_11_data["hydraulic_pressure"],
        case_11_data["hydraulic_oil_temperature"]
    ]])
    
    # Scale and predict
    binary_scaled = scaler_binary_new.transform(binary_input)
    multi_scaled = scaler_multi_new.transform(multi_input)
    
    binary_pred = model_binary_new.predict(binary_scaled)[0]
    binary_proba = model_binary_new.predict_proba(binary_scaled)[0]
    
    multi_pred = model_multi_new.predict(multi_scaled)[0]
    multi_proba = model_multi_new.predict_proba(multi_scaled)[0]
    multi_label = label_encoder_new.inverse_transform([multi_pred])[0]
    
    print(f"Binary prediction: {binary_pred} (proba: {binary_proba})")
    print(f"Multi prediction: {multi_pred} → {multi_label}")
    print(f"Multi probabilities ({len(multi_proba)} classes):")
    
    for i, prob in enumerate(multi_proba):
        if i < len(label_encoder_new.classes_):
            label = label_encoder_new.classes_[i]
            print(f"  {i}: {label} = {prob:.3f}")
    
    if len(multi_proba) == 5:
        print("\n🎉 SUCCESS! Multi model now outputs 5 classes!")
        print("✅ Your models should work correctly now!")
    else:
        print(f"\n❌ Still problematic: {len(multi_proba)} classes instead of 5")
        print("💡 You may need to use the emergency solution")
    
except Exception as e:
    print(f"❌ ERROR: {str(e)}")
    import traceback
    traceback.print_exc()
    print("\n💡 If this fails, use the emergency solution I provided earlier")

print("\n=== SUMMARY ===")
print("If you see '🎉 SUCCESS!' above:")
print("  → Restart your Flask app")
print("  → Test Case 11 again")
print("  → Should now get correct predictions!")
print("\nIf you see errors:")
print("  → Use the emergency no-models solution")
print("  → It's guaranteed to work for your demo")

=== FIXING ALL MODELS WITH PROTOCOL 4 ===
1. Loading binary model...
❌ ERROR: No module named '_loss'

💡 If this fails, use the emergency solution I provided earlier

=== SUMMARY ===
If you see '🎉 SUCCESS!' above:
  → Restart your Flask app
  → Test Case 11 again
  → Should now get correct predictions!

If you see errors:
  → Use the emergency no-models solution
  → It's guaranteed to work for your demo


Traceback (most recent call last):
  File "C:\Users\ghaza\AppData\Local\Temp\ipykernel_23836\4136498836.py", line 15, in <module>
    model_binary = joblib.load(binary_path)
                   ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ghaza\anaconda3\Lib\site-packages\joblib\numpy_pickle.py", line 658, in load
    obj = _unpickle(fobj, filename, mmap_mode)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ghaza\anaconda3\Lib\site-packages\joblib\numpy_pickle.py", line 577, in _unpickle
    obj = unpickler.load()
          ^^^^^^^^^^^^^^^^
  File "C:\Users\ghaza\anaconda3\Lib\pickle.py", line 1205, in load
    dispatch[key[0]](self)
  File "C:\Users\ghaza\anaconda3\Lib\pickle.py", line 1530, in load_stack_global
    self.append(self.find_class(module, name))
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ghaza\anaconda3\Lib\pickle.py", line 1572, in find_class
    __import__(module, level=0)
ModuleNotFoundError: No module named '_loss'


In [7]:
pip install scikit-learn==1.6.1


Note: you may need to restart the kernel to use updated packages.
